# NB15: Layer 2 Scaling Simulation [ADVANCED]

**Explore Layer 2 blockchain scaling solutions through interactive simulations**

---

## Learning Objectives

By the end of this notebook, you will:

1. **Understand State Channels**: Simulate bi-directional payment channels with off-chain updates
2. **Explore Rollups**: Build simple rollup batching and compression mechanisms
3. **Detect Fraud**: Implement optimistic rollup fraud proofs and challenge periods
4. **Compare Costs**: Analyze gas savings between Layer 1 and Layer 2 solutions
5. **Apply Knowledge**: Complete challenges extending L2 concepts

**Prerequisites**: Understanding of blockchain basics, transactions, and gas costs

## Setup and Imports

We'll use standard Python libraries to simulate Layer 2 scaling mechanisms.

In [ ]:
import hashlib
import time
import json
import random
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional
import matplotlib.pyplot as plt

print("✓ All libraries imported successfully")
print("Ready to simulate Layer 2 scaling solutions!")

## Introduction to Layer 2 Scaling

**The Scalability Trilemma**: Blockchains struggle to achieve decentralization, security, AND scalability simultaneously.

**Layer 2 Solutions** move transaction execution off-chain while inheriting Layer 1 security:

- **State Channels**: Off-chain bilateral agreements, settle final state on-chain
- **Rollups**: Batch many transactions, post compressed data to L1
  - **Optimistic Rollups**: Assume validity, allow challenges
  - **ZK-Rollups**: Cryptographic validity proofs
- **Sidechains**: Independent chains with bridges to main chain

**Key Benefit**: 10x to 100x transaction throughput increase with lower fees

---

# Section 1: State Channels Simulation

## What Are State Channels?

**Simple Analogy**: State channels are like keeping a running tab with a friend at a coffee shop. Instead of paying for each coffee individually (which costs transaction fees each time), you keep track of what you owe on paper. At the end of the month, you settle up with one final payment.

**Technical Concept**: Two parties lock funds on-chain, perform unlimited off-chain updates, settle final state.

**How It Works**:
1. **Open**: Both parties deposit funds into a smart contract (like putting money in a shared envelope)
2. **Transact**: They exchange signed messages updating balances (writing new totals on the envelope)
3. **Close**: Submit final state to blockchain for settlement (opening the envelope and distributing funds)

**Real-world Example**: Lightning Network (Bitcoin), Raiden Network (Ethereum)

**Key Benefit**: Thousands of transactions with only 2 blockchain transactions needed!

> **Cross-reference**: For detailed explanation of state channels and Layer 2 concepts, see Topic 3.5 in the course materials.

In [ ]:
@dataclass
class ChannelState:
    """Represents a snapshot of payment channel state."""
    channel_id: str
    alice_balance: float
    bob_balance: float
    nonce: int  # Sequence number to prevent replay attacks
    timestamp: float
    
    def to_hash(self) -> str:
        """Create cryptographic commitment to this state."""
        state_str = f"{self.channel_id}{self.alice_balance}{self.bob_balance}{self.nonce}"
        return hashlib.sha256(state_str.encode()).hexdigest()
    
    def __repr__(self):
        return (f"State #{self.nonce}: Alice={self.alice_balance:.2f}, "
                f"Bob={self.bob_balance:.2f}")

print("✓ ChannelState dataclass defined")

In [ ]:
class PaymentChannel:
    """Simulates a bi-directional payment channel."""
    
    def __init__(self, channel_id: str, alice_deposit: float, bob_deposit: float):
        self.channel_id = channel_id
        self.initial_alice = alice_deposit
        self.initial_bob = bob_deposit
        self.current_state = ChannelState(
            channel_id=channel_id,
            alice_balance=alice_deposit,
            bob_balance=bob_deposit,
            nonce=0,
            timestamp=time.time()
        )
        self.state_history = [self.current_state]
        self.is_open = True
        
        print(f"\n💰 Channel Opened: {self.channel_id}")
        print(f"   Alice deposits: {alice_deposit:.2f} ETH")
        print(f"   Bob deposits: {bob_deposit:.2f} ETH")
        print(f"   Total locked: {alice_deposit + bob_deposit:.2f} ETH")
    
    def update_state(self, amount: float, direction: str) -> ChannelState:
        """Off-chain state update (signed by both parties in reality).
        
        Args:
            amount: Amount to transfer
            direction: 'alice_to_bob' or 'bob_to_alice'
        """
        if not self.is_open:
            raise ValueError("Channel is closed")
        
        new_alice = self.current_state.alice_balance
        new_bob = self.current_state.bob_balance
        
        if direction == 'alice_to_bob':
            if new_alice < amount:
                raise ValueError(f"Insufficient balance: Alice has {new_alice:.2f}")
            new_alice -= amount
            new_bob += amount
            sender, receiver = "Alice", "Bob"
        elif direction == 'bob_to_alice':
            if new_bob < amount:
                raise ValueError(f"Insufficient balance: Bob has {new_bob:.2f}")
            new_bob -= amount
            new_alice += amount
            sender, receiver = "Bob", "Alice"
        else:
            raise ValueError("Direction must be 'alice_to_bob' or 'bob_to_alice'")
        
        # Create new state with incremented nonce
        new_state = ChannelState(
            channel_id=self.channel_id,
            alice_balance=new_alice,
            bob_balance=new_bob,
            nonce=self.current_state.nonce + 1,
            timestamp=time.time()
        )
        
        self.current_state = new_state
        self.state_history.append(new_state)
        
        print(f"\n📝 Off-chain update #{new_state.nonce}:")
        print(f"   {sender} → {receiver}: {amount:.2f} ETH")
        print(f"   New balances: {new_state}")
        print(f"   State hash: {new_state.to_hash()[:16]}...")
        
        return new_state
    
    def close_channel(self) -> dict:
        """Submit final state to L1 (simulated)."""
        if not self.is_open:
            raise ValueError("Channel already closed")
        
        self.is_open = False
        final_state = self.current_state
        
        settlement = {
            'channel_id': self.channel_id,
            'total_updates': final_state.nonce,
            'alice_initial': self.initial_alice,
            'alice_final': final_state.alice_balance,
            'alice_net': final_state.alice_balance - self.initial_alice,
            'bob_initial': self.initial_bob,
            'bob_final': final_state.bob_balance,
            'bob_net': final_state.bob_balance - self.initial_bob,
            'final_state_hash': final_state.to_hash()
        }
        
        print(f"\n🔒 Channel Closed: {self.channel_id}")
        print(f"   Total off-chain updates: {settlement['total_updates']}")
        print(f"   Alice: {settlement['alice_initial']:.2f} → {settlement['alice_final']:.2f} "
              f"({settlement['alice_net']:+.2f} ETH)")
        print(f"   Bob: {settlement['bob_initial']:.2f} → {settlement['bob_final']:.2f} "
              f"({settlement['bob_net']:+.2f} ETH)")
        print(f"   L1 transactions: 2 (open + close)")
        print(f"   L1 gas saved: {settlement['total_updates']} transactions worth!")
        
        return settlement

print("✓ PaymentChannel class defined")

### Example: Coffee Shop Payment Channel

Alice visits Bob's coffee shop daily. Instead of paying on-chain each time (expensive!), they open a channel.

In [ ]:
# Alice deposits 10 ETH, Bob deposits 0 (he's the merchant)
coffee_channel = PaymentChannel(
    channel_id="alice-bob-coffee",
    alice_deposit=10.0,
    bob_deposit=0.0
)

In [ ]:
# Day 1: Cappuccino (0.05 ETH)
coffee_channel.update_state(0.05, 'alice_to_bob')

# Day 2: Latte (0.04 ETH)
coffee_channel.update_state(0.04, 'alice_to_bob')

# Day 3: Espresso (0.03 ETH)
coffee_channel.update_state(0.03, 'alice_to_bob')

# Day 4: Refund for cold coffee (0.04 ETH back to Alice)
coffee_channel.update_state(0.04, 'bob_to_alice')

# Day 5: Americano (0.03 ETH)
coffee_channel.update_state(0.03, 'alice_to_bob')

In [ ]:
# End of week: Close the channel
settlement = coffee_channel.close_channel()

### Visualization: State Channel Evolution

In [ ]:
def visualize_channel_states(channel: PaymentChannel):
    """Plot balance changes over channel lifetime."""
    nonces = [state.nonce for state in channel.state_history]
    alice_balances = [state.alice_balance for state in channel.state_history]
    bob_balances = [state.bob_balance for state in channel.state_history]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Line chart: Balance evolution
    ax1.plot(nonces, alice_balances, marker='o', label='Alice', linewidth=2, markersize=8)
    ax1.plot(nonces, bob_balances, marker='s', label='Bob', linewidth=2, markersize=8)
    ax1.set_xlabel('State Update (Nonce)', fontsize=12)
    ax1.set_ylabel('Balance (ETH)', fontsize=12)
    ax1.set_title('Payment Channel Balance Evolution', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Bar chart: Initial vs Final
    parties = ['Alice', 'Bob']
    initial = [channel.initial_alice, channel.initial_bob]
    final = [channel.current_state.alice_balance, channel.current_state.bob_balance]
    
    x = range(len(parties))
    width = 0.35
    ax2.bar([i - width/2 for i in x], initial, width, label='Initial', alpha=0.8)
    ax2.bar([i + width/2 for i in x], final, width, label='Final', alpha=0.8)
    ax2.set_xlabel('Party', fontsize=12)
    ax2.set_ylabel('Balance (ETH)', fontsize=12)
    ax2.set_title('Initial vs Final Balances', fontsize=14, fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels(parties)
    ax2.legend(fontsize=11)
    ax2.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Channel Statistics:")
    print(f"   Total state updates: {len(channel.state_history) - 1}")
    print(f"   L1 transactions needed without channel: {len(channel.state_history) + 1}")
    print(f"   Actual L1 transactions: 2 (open + close)")
    print(f"   Gas efficiency: {((len(channel.state_history) + 1 - 2) / (len(channel.state_history) + 1)) * 100:.1f}% reduction")

visualize_channel_states(coffee_channel)

**Key Insight**: 5 off-chain payments with only 2 on-chain transactions = 71% gas savings!

---

# Section 2: Simple Rollup Concept

## What Are Rollups?

**Simple Analogy**: Batching = combining many transactions into one. Like sending one big envelope with 100 letters instead of 100 separate envelopes - much cheaper postage!

**Rollup Philosophy**: Bundle many transactions, post compressed batch to L1

**Key Components**:
1. **Batch**: Collect pending transactions (like gathering letters to mail)
2. **Compute**: Calculate new state root using a Merkle tree (a compact proof of all balances)
3. **Post**: Submit compressed data + state root to L1 (mail the big envelope)
4. **Availability**: L1 stores data so anyone can reconstruct state if needed

**Why This Works**: Instead of 100 separate blockchain transactions (expensive!), we submit 1 batch transaction containing all 100 compressed together (cheap!).

In [ ]:
class SimpleRollup:
    """Simulates basic rollup batching and compression."""
    
    def __init__(self):
        self.pending_transactions = []
        self.state_root = self._compute_empty_root()
        self.batches = []
        self.account_balances = {}  # Simplified state
        
        print("\n🚀 Simple Rollup Initialized")
        print(f"   Initial state root: {self.state_root[:16]}...")
    
    def _compute_empty_root(self) -> str:
        """Compute hash of empty state."""
        return hashlib.sha256(b"empty").hexdigest()
    
    def add_transaction(self, tx: dict):
        """Add transaction to pending batch."""
        tx['tx_id'] = hashlib.sha256(json.dumps(tx, sort_keys=True).encode()).hexdigest()[:16]
        self.pending_transactions.append(tx)
        print(f"\n📥 Transaction added to pending batch:")
        print(f"   From: {tx['from']}")
        print(f"   To: {tx['to']}")
        print(f"   Amount: {tx['amount']} ETH")
        print(f"   TX ID: {tx['tx_id']}")
        print(f"   Pending batch size: {len(self.pending_transactions)}")
    
    def create_batch(self) -> dict:
        """Batch transactions and compute new state root."""
        if not self.pending_transactions:
            raise ValueError("No pending transactions to batch")
        
        # Execute transactions and update state
        for tx in self.pending_transactions:
            sender = tx['from']
            receiver = tx['to']
            amount = tx['amount']
            
            # Update balances (simplified, no validation)
            self.account_balances[sender] = self.account_balances.get(sender, 0) - amount
            self.account_balances[receiver] = self.account_balances.get(receiver, 0) + amount
        
        # Compute new state root
        state_str = json.dumps(self.account_balances, sort_keys=True)
        new_state_root = hashlib.sha256(state_str.encode()).hexdigest()
        
        # Create compressed batch
        batch = {
            'batch_id': len(self.batches) + 1,
            'num_transactions': len(self.pending_transactions),
            'old_state_root': self.state_root,
            'new_state_root': new_state_root,
            'transactions': self.pending_transactions.copy(),
            'timestamp': time.time()
        }
        
        # Calculate compression
        original_size = sum(len(json.dumps(tx)) for tx in self.pending_transactions)
        compressed_size = len(new_state_root) + 64  # State root + batch metadata
        
        batch['original_size_bytes'] = original_size
        batch['compressed_size_bytes'] = compressed_size
        batch['compression_ratio'] = original_size / compressed_size if compressed_size > 0 else 0
        
        self.state_root = new_state_root
        self.batches.append(batch)
        self.pending_transactions = []
        
        print(f"\n📦 Batch #{batch['batch_id']} Created:")
        print(f"   Transactions batched: {batch['num_transactions']}")
        print(f"   Old state root: {batch['old_state_root'][:16]}...")
        print(f"   New state root: {batch['new_state_root'][:16]}...")
        print(f"   Original data size: {batch['original_size_bytes']} bytes")
        print(f"   Compressed size: {batch['compressed_size_bytes']} bytes")
        print(f"   Compression ratio: {batch['compression_ratio']:.2f}x")
        
        return batch
    
    def post_to_l1(self, batch: dict) -> dict:
        """Simulate posting batch to L1 (single transaction)."""
        l1_tx = {
            'type': 'rollup_batch',
            'batch_id': batch['batch_id'],
            'state_root': batch['new_state_root'],
            'num_l2_transactions': batch['num_transactions'],
            'data_size_bytes': batch['compressed_size_bytes'],
            'gas_used': 21000 + (batch['compressed_size_bytes'] * 16),  # Base + calldata
            'timestamp': time.time()
        }
        
        print(f"\n⛓️  Batch Posted to L1:")
        print(f"   L1 transaction type: Rollup batch")
        print(f"   L2 transactions included: {l1_tx['num_l2_transactions']}")
        print(f"   L1 gas used: {l1_tx['gas_used']:,}")
        print(f"   Gas per L2 tx: {l1_tx['gas_used'] / l1_tx['num_l2_transactions']:.0f}")
        print(f"   L1 gas savings: {((21000 * l1_tx['num_l2_transactions'] - l1_tx['gas_used']) / (21000 * l1_tx['num_l2_transactions'])) * 100:.1f}%")
        
        return l1_tx

print("✓ SimpleRollup class defined")

### Example: DEX Trading on Rollup

Multiple users trade on a decentralized exchange. Trades are batched together.

In [ ]:
# Initialize rollup
dex_rollup = SimpleRollup()

In [ ]:
# Add multiple DEX trades
dex_rollup.add_transaction({'from': 'alice', 'to': 'bob', 'amount': 1.5})
dex_rollup.add_transaction({'from': 'bob', 'to': 'charlie', 'amount': 0.8})
dex_rollup.add_transaction({'from': 'charlie', 'to': 'diana', 'amount': 2.3})
dex_rollup.add_transaction({'from': 'diana', 'to': 'alice', 'amount': 1.1})
dex_rollup.add_transaction({'from': 'alice', 'to': 'charlie', 'amount': 0.5})

In [ ]:
# Create batch from pending transactions
batch_1 = dex_rollup.create_batch()

In [ ]:
# Post batch to L1
l1_tx_1 = dex_rollup.post_to_l1(batch_1)

### Second Batch: More Trades

In [ ]:
# Add more transactions
for i in range(10):
    users = ['alice', 'bob', 'charlie', 'diana', 'eve']
    sender = random.choice(users)
    receiver = random.choice([u for u in users if u != sender])
    amount = round(random.uniform(0.1, 2.0), 2)
    dex_rollup.add_transaction({'from': sender, 'to': receiver, 'amount': amount})

print(f"\n✓ Added 10 random transactions to pending batch")

In [ ]:
# Create and post second batch
batch_2 = dex_rollup.create_batch()
l1_tx_2 = dex_rollup.post_to_l1(batch_2)

### Visualization: Rollup Efficiency

In [ ]:
def visualize_rollup_efficiency(rollup: SimpleRollup):
    """Visualize compression and gas savings across batches."""
    if not rollup.batches:
        print("No batches to visualize")
        return
    
    batch_ids = [b['batch_id'] for b in rollup.batches]
    compressions = [b['compression_ratio'] for b in rollup.batches]
    tx_counts = [b['num_transactions'] for b in rollup.batches]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Compression ratios
    ax1.bar(batch_ids, compressions, color='steelblue', alpha=0.8)
    ax1.set_xlabel('Batch ID', fontsize=12)
    ax1.set_ylabel('Compression Ratio', fontsize=12)
    ax1.set_title('Data Compression per Batch', fontsize=14, fontweight='bold')
    ax1.grid(True, axis='y', alpha=0.3)
    
    # Transactions per batch
    ax2.bar(batch_ids, tx_counts, color='coral', alpha=0.8)
    ax2.set_xlabel('Batch ID', fontsize=12)
    ax2.set_ylabel('Number of Transactions', fontsize=12)
    ax2.set_title('Transactions Batched', fontsize=14, fontweight='bold')
    ax2.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    total_l2_txs = sum(tx_counts)
    total_l1_txs = len(rollup.batches)
    
    print(f"\n📊 Rollup Statistics:")
    print(f"   Total L2 transactions: {total_l2_txs}")
    print(f"   Total L1 transactions: {total_l1_txs}")
    print(f"   Efficiency: {total_l2_txs / total_l1_txs:.1f}x transactions per L1 batch")
    print(f"   Average compression: {sum(compressions) / len(compressions):.2f}x")

visualize_rollup_efficiency(dex_rollup)

---

# Section 3: Optimistic Rollup with Fraud Proofs

**Optimistic Assumption**: All transactions are valid unless proven otherwise

**Challenge Period**: 7-day window where anyone can submit fraud proof

**Fraud Proof**: Cryptographic evidence that a state transition was invalid

In [ ]:
class OptimisticRollup(SimpleRollup):
    """Extends SimpleRollup with optimistic fraud proofs."""
    
    def __init__(self, challenge_period_days: int = 7):
        super().__init__()
        self.challenge_period_days = challenge_period_days
        self.pending_states = []  # States in challenge period
        self.finalized_states = []  # States past challenge period
        self.challenges = []
        
        print(f"\n🔍 Optimistic Rollup Initialized")
        print(f"   Challenge period: {challenge_period_days} days")
        print(f"   Fraud detection: Active")
    
    def submit_state(self, claimed_state: dict) -> dict:
        """Submit state claim (can be challenged during period)."""
        submission = {
            'state_id': len(self.pending_states) + len(self.finalized_states) + 1,
            'claimed_state': claimed_state,
            'submitted_at': time.time(),
            'finalized_at': None,
            'status': 'pending',
            'challenged': False
        }
        
        self.pending_states.append(submission)
        
        print(f"\n📤 State Submitted:")
        print(f"   State ID: {submission['state_id']}")
        print(f"   Status: {submission['status'].upper()}")
        print(f"   Challenge window: {self.challenge_period_days} days")
        print(f"   State root: {claimed_state.get('state_root', 'N/A')[:16]}...")
        
        return submission
    
    def challenge(self, state_id: int, fraud_proof: dict) -> dict:
        """Challenge a pending state with fraud proof."""
        # Find the state
        target_state = None
        for state in self.pending_states:
            if state['state_id'] == state_id:
                target_state = state
                break
        
        if not target_state:
            return {'success': False, 'reason': 'State not found or already finalized'}
        
        if target_state['challenged']:
            return {'success': False, 'reason': 'State already challenged'}
        
        # Verify fraud proof (simplified: check if claimed differs from expected)
        is_valid_challenge = self._verify_fraud_proof(target_state, fraud_proof)
        
        challenge_result = {
            'challenge_id': len(self.challenges) + 1,
            'state_id': state_id,
            'fraud_proof': fraud_proof,
            'is_valid': is_valid_challenge,
            'timestamp': time.time()
        }
        
        self.challenges.append(challenge_result)
        
        if is_valid_challenge:
            target_state['challenged'] = True
            target_state['status'] = 'rejected'
            self.pending_states.remove(target_state)
            
            print(f"\n⚠️  Challenge SUCCESSFUL:")
            print(f"   State ID: {state_id}")
            print(f"   Challenge ID: {challenge_result['challenge_id']}")
            print(f"   Fraud detected: {fraud_proof.get('reason', 'Invalid state transition')}")
            print(f"   Action: State REJECTED and removed")
            print(f"   Proposer slashed: ❌")
        else:
            print(f"\n✅ Challenge FAILED:")
            print(f"   State ID: {state_id}")
            print(f"   Challenge ID: {challenge_result['challenge_id']}")
            print(f"   State is valid, challenge dismissed")
            print(f"   Challenger slashed: ❌")
        
        return challenge_result
    
    def _verify_fraud_proof(self, state_submission: dict, fraud_proof: dict) -> bool:
        """Verify if fraud proof is valid (simplified)."""
        claimed = state_submission['claimed_state']
        expected = fraud_proof.get('expected_state', {})
        
        # Compare state roots
        if 'state_root' in claimed and 'state_root' in expected:
            return claimed['state_root'] != expected['state_root']
        
        return False
    
    def finalize_states(self, simulated_days_passed: int = 7):
        """Finalize states past challenge period."""
        current_time = time.time()
        challenge_period_seconds = self.challenge_period_days * 24 * 3600
        
        # For simulation, use parameter
        if simulated_days_passed >= self.challenge_period_days:
            finalized_count = 0
            for state in self.pending_states[:]:
                if not state['challenged']:
                    state['status'] = 'finalized'
                    state['finalized_at'] = current_time
                    self.finalized_states.append(state)
                    self.pending_states.remove(state)
                    finalized_count += 1
            
            print(f"\n✅ States Finalized:")
            print(f"   Days passed: {simulated_days_passed}")
            print(f"   States finalized: {finalized_count}")
            print(f"   Pending states: {len(self.pending_states)}")
            print(f"   Total finalized: {len(self.finalized_states)}")
        else:
            print(f"\n⏳ Still in challenge period:")
            print(f"   Days passed: {simulated_days_passed}")
            print(f"   Days remaining: {self.challenge_period_days - simulated_days_passed}")

print("✓ OptimisticRollup class defined")

### Example: Valid State Submission

### What We're Doing Next

We'll simulate the Optimistic Rollup challenge system:
1. **Submit** a valid state (transaction batch results)
2. **Challenge** it with an invalid fraud proof (should fail)
3. **Wait** 7 days (simulated) for the challenge period to end
4. **Finalize** the state since no valid fraud was found

Watch how the system protects against invalid challenges!

### What We're Doing Next

We'll simulate the Optimistic Rollup challenge system:
1. **Submit** a valid state (transaction batch results)
2. **Challenge** it with an invalid fraud proof (should fail)
3. **Wait** 7 days (simulated) for the challenge period to end
4. **Finalize** the state since no valid fraud was found

Watch how the system protects against invalid challenges!

In [ ]:
# Initialize optimistic rollup
op_rollup = OptimisticRollup(challenge_period_days=7)

# Submit a valid state
valid_state = {
    'state_root': hashlib.sha256(b"valid_state_123").hexdigest(),
    'num_transactions': 50,
    'total_value': 125.5
}

submission_1 = op_rollup.submit_state(valid_state)

In [ ]:
# Try to challenge with invalid proof (state is actually valid)
invalid_fraud_proof = {
    'reason': 'Invalid state transition',
    'expected_state': {
        'state_root': hashlib.sha256(b"valid_state_123").hexdigest(),  # Same as claimed
        'num_transactions': 50
    }
}

challenge_result_1 = op_rollup.challenge(submission_1['state_id'], invalid_fraud_proof)

In [ ]:
# Wait 7 days (simulated) and finalize
op_rollup.finalize_states(simulated_days_passed=7)

### Example: Fraudulent State Detection

### Detecting Fraud

Now let's see what happens when someone tries to cheat:
- A malicious actor submits a **fraudulent state** (claiming balances that don't match reality)
- An honest validator **detects the fraud** by comparing to the correct state
- They submit a **fraud proof** showing the mismatch
- The system **rejects** the fraudulent state and **slashes** (penalizes) the malicious actor

This is the core security mechanism of Optimistic Rollups!

### Detecting Fraud

Now let's see what happens when someone tries to cheat:
- A malicious actor submits a **fraudulent state** (claiming balances that don't match reality)
- An honest validator **detects the fraud** by comparing to the correct state
- They submit a **fraud proof** showing the mismatch
- The system **rejects** the fraudulent state and **slashes** (penalizes) the malicious actor

This is the core security mechanism of Optimistic Rollups!

In [ ]:
# Malicious actor submits fraudulent state
fraudulent_state = {
    'state_root': hashlib.sha256(b"fraudulent_state_999").hexdigest(),
    'num_transactions': 100,
    'total_value': 999999.0  # Inflated value!
}

submission_2 = op_rollup.submit_state(fraudulent_state)

In [ ]:
# Honest validator detects fraud and submits proof
valid_fraud_proof = {
    'reason': 'State root mismatch - invalid balance inflation',
    'expected_state': {
        'state_root': hashlib.sha256(b"correct_state_100").hexdigest(),  # Different!
        'num_transactions': 100,
        'total_value': 250.0  # Correct value
    },
    'proof_tx_index': 42,
    'invalid_transition': 'Balance increased without deposit'
}

challenge_result_2 = op_rollup.challenge(submission_2['state_id'], valid_fraud_proof)

### Visualization: Challenge System

In [ ]:
def visualize_challenge_system(rollup: OptimisticRollup):
    """Visualize state submissions and challenge outcomes."""
    all_states = rollup.pending_states + rollup.finalized_states
    
    if not all_states:
        print("No states to visualize")
        return
    
    statuses = {}
    for state in all_states:
        status = state['status']
        statuses[status] = statuses.get(status, 0) + 1
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Pie chart: State status distribution
    colors = {'pending': 'gold', 'finalized': 'green', 'rejected': 'red'}
    ax1.pie(statuses.values(), labels=statuses.keys(), autopct='%1.1f%%',
            colors=[colors.get(k, 'gray') for k in statuses.keys()],
            startangle=90)
    ax1.set_title('State Status Distribution', fontsize=14, fontweight='bold')
    
    # Bar chart: Challenge outcomes
    if rollup.challenges:
        valid_challenges = sum(1 for c in rollup.challenges if c['is_valid'])
        invalid_challenges = len(rollup.challenges) - valid_challenges
        
        ax2.bar(['Valid Fraud\nProofs', 'Invalid\nChallenges'],
                [valid_challenges, invalid_challenges],
                color=['crimson', 'lightgray'], alpha=0.8)
        ax2.set_ylabel('Count', fontsize=12)
        ax2.set_title('Challenge Outcomes', fontsize=14, fontweight='bold')
        ax2.grid(True, axis='y', alpha=0.3)
    else:
        ax2.text(0.5, 0.5, 'No challenges submitted', ha='center', va='center',
                fontsize=14, transform=ax2.transAxes)
        ax2.set_title('Challenge Outcomes', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Optimistic Rollup Statistics:")
    print(f"   Total states submitted: {len(all_states)}")
    print(f"   Finalized states: {len(rollup.finalized_states)}")
    print(f"   Pending states: {len(rollup.pending_states)}")
    print(f"   Total challenges: {len(rollup.challenges)}")
    if rollup.challenges:
        valid_rate = (valid_challenges / len(rollup.challenges)) * 100
        print(f"   Valid fraud proofs: {valid_challenges} ({valid_rate:.1f}%)")

visualize_challenge_system(op_rollup)

---

# Section 4: Gas Cost Comparison

## Why Gas Costs Matter

**The Bottom Line**: How much cheaper is Layer 2?

**What is Gas?** Gas is the fee you pay to use the Ethereum network. Think of it like paying for electricity - the more computation your transaction requires, the more gas you pay.

**Why L2 Saves Money**: These gas costs show why Layer 2 solutions save money. Processing transactions on L2 is much cheaper than on the main Ethereum network (L1) because:
- L2 batches many transactions together
- Only one transaction is posted to expensive L1
- The cost is split among all transactions in the batch

Let's calculate real gas costs for L1 vs L2 scenarios and see the dramatic savings!

In [ ]:
def calculate_l1_cost(num_transactions: int, gas_price_gwei: int = 30) -> dict:
    """Calculate Layer 1 gas costs.
    
    Args:
        num_transactions: Number of transactions
        gas_price_gwei: Gas price in Gwei
    
    Returns:
        Dictionary with cost breakdown
    """
    GAS_PER_TX = 21000  # Simple ETH transfer
    GWEI_TO_ETH = 1e-9
    
    total_gas = num_transactions * GAS_PER_TX
    total_cost_eth = total_gas * gas_price_gwei * GWEI_TO_ETH
    cost_per_tx_eth = total_cost_eth / num_transactions
    
    return {
        'num_transactions': num_transactions,
        'gas_per_tx': GAS_PER_TX,
        'total_gas': total_gas,
        'gas_price_gwei': gas_price_gwei,
        'total_cost_eth': total_cost_eth,
        'cost_per_tx_eth': cost_per_tx_eth,
        'total_cost_usd': total_cost_eth * 2000  # Assume 1 ETH = $2000
    }

def calculate_l2_cost(num_transactions: int, gas_price_gwei: int = 30,
                      batch_size: int = 100) -> dict:
    """Calculate Layer 2 gas costs (batched).
    
    Args:
        num_transactions: Number of L2 transactions
        gas_price_gwei: L1 gas price in Gwei
        batch_size: Transactions per batch
    
    Returns:
        Dictionary with cost breakdown
    """
    GWEI_TO_ETH = 1e-9
    BATCH_BASE_GAS = 21000
    CALLDATA_GAS_PER_BYTE = 16
    BYTES_PER_TX_COMPRESSED = 12  # Heavily compressed transaction data
    
    num_batches = (num_transactions + batch_size - 1) // batch_size
    
    # Gas per batch
    calldata_bytes_per_batch = min(batch_size, num_transactions % batch_size or batch_size) * BYTES_PER_TX_COMPRESSED
    gas_per_batch = BATCH_BASE_GAS + (calldata_bytes_per_batch * CALLDATA_GAS_PER_BYTE)
    
    # Total costs
    total_l1_gas = num_batches * gas_per_batch
    total_cost_eth = total_l1_gas * gas_price_gwei * GWEI_TO_ETH
    cost_per_tx_eth = total_cost_eth / num_transactions
    
    return {
        'num_transactions': num_transactions,
        'batch_size': batch_size,
        'num_batches': num_batches,
        'gas_per_batch': gas_per_batch,
        'total_l1_gas': total_l1_gas,
        'gas_price_gwei': gas_price_gwei,
        'total_cost_eth': total_cost_eth,
        'cost_per_tx_eth': cost_per_tx_eth,
        'total_cost_usd': total_cost_eth * 2000
    }

print("✓ Gas calculation functions defined")

### Example: Compare 1000 Transactions

In [ ]:
# Calculate costs for 1000 transactions
l1_costs = calculate_l1_cost(1000, gas_price_gwei=30)
l2_costs = calculate_l2_cost(1000, gas_price_gwei=30, batch_size=100)

print("\n⛓️  LAYER 1 (Direct):")
print(f"   Transactions: {l1_costs['num_transactions']}")
print(f"   Gas per tx: {l1_costs['gas_per_tx']:,}")
print(f"   Total gas: {l1_costs['total_gas']:,}")
print(f"   Total cost: {l1_costs['total_cost_eth']:.4f} ETH (${l1_costs['total_cost_usd']:.2f})")
print(f"   Cost per tx: {l1_costs['cost_per_tx_eth']:.6f} ETH")

print("\n🚀 LAYER 2 (Rollup):")
print(f"   Transactions: {l2_costs['num_transactions']}")
print(f"   Batch size: {l2_costs['batch_size']}")
print(f"   Number of batches: {l2_costs['num_batches']}")
print(f"   Gas per batch: {l2_costs['gas_per_batch']:,}")
print(f"   Total L1 gas: {l2_costs['total_l1_gas']:,}")
print(f"   Total cost: {l2_costs['total_cost_eth']:.4f} ETH (${l2_costs['total_cost_usd']:.2f})")
print(f"   Cost per tx: {l2_costs['cost_per_tx_eth']:.6f} ETH")

savings_eth = l1_costs['total_cost_eth'] - l2_costs['total_cost_eth']
savings_pct = (savings_eth / l1_costs['total_cost_eth']) * 100

print("\n💰 SAVINGS:")
print(f"   Absolute: {savings_eth:.4f} ETH (${savings_eth * 2000:.2f})")
print(f"   Percentage: {savings_pct:.1f}%")
print(f"   Cost reduction: {l1_costs['cost_per_tx_eth'] / l2_costs['cost_per_tx_eth']:.1f}x cheaper per transaction")

### Comprehensive Cost Analysis

In [ ]:
def visualize_savings(transactions_range: list, gas_price_gwei: int = 30):
    """Plot L1 vs L2 costs across transaction volumes."""
    l1_costs_list = []
    l2_costs_list = []
    
    for num_tx in transactions_range:
        l1 = calculate_l1_cost(num_tx, gas_price_gwei)
        l2 = calculate_l2_cost(num_tx, gas_price_gwei)
        l1_costs_list.append(l1['total_cost_eth'])
        l2_costs_list.append(l2['total_cost_eth'])
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Line chart: Absolute costs
    ax1.plot(transactions_range, l1_costs_list, marker='o', label='Layer 1',
             linewidth=2, markersize=8, color='red')
    ax1.plot(transactions_range, l2_costs_list, marker='s', label='Layer 2',
             linewidth=2, markersize=8, color='green')
    ax1.set_xlabel('Number of Transactions', fontsize=12)
    ax1.set_ylabel('Total Cost (ETH)', fontsize=12)
    ax1.set_title('L1 vs L2 Total Costs', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    
    # Bar chart: Percentage savings
    savings_pct = [((l1 - l2) / l1 * 100) if l1 > 0 else 0
                   for l1, l2 in zip(l1_costs_list, l2_costs_list)]
    
    ax2.bar(range(len(transactions_range)), savings_pct, color='teal', alpha=0.8)
    ax2.set_xlabel('Transaction Volume', fontsize=12)
    ax2.set_ylabel('Savings (%)', fontsize=12)
    ax2.set_title('L2 Gas Savings', fontsize=14, fontweight='bold')
    ax2.set_xticks(range(len(transactions_range)))
    ax2.set_xticklabels([str(n) for n in transactions_range], rotation=45)
    ax2.grid(True, axis='y', alpha=0.3)
    ax2.axhline(y=90, color='red', linestyle='--', alpha=0.5, label='90% savings')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Cost Analysis Summary (Gas price: {gas_price_gwei} Gwei):")
    for i, num_tx in enumerate(transactions_range):
        print(f"   {num_tx:>5} txs: L1={l1_costs_list[i]:.4f} ETH, "
              f"L2={l2_costs_list[i]:.4f} ETH, Savings={savings_pct[i]:.1f}%")

# Analyze different transaction volumes
visualize_savings([1, 10, 100, 1000, 10000], gas_price_gwei=30)

### Gas Price Sensitivity Analysis

In [ ]:
def compare_gas_prices(num_transactions: int = 1000):
    """Compare costs at different gas prices."""
    gas_prices = [10, 30, 50, 100, 200]  # Gwei
    l1_costs = []
    l2_costs = []
    
    for gp in gas_prices:
        l1 = calculate_l1_cost(num_transactions, gp)
        l2 = calculate_l2_cost(num_transactions, gp)
        l1_costs.append(l1['total_cost_usd'])
        l2_costs.append(l2['total_cost_usd'])
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = range(len(gas_prices))
    width = 0.35
    
    ax.bar([i - width/2 for i in x], l1_costs, width, label='Layer 1', alpha=0.8, color='red')
    ax.bar([i + width/2 for i in x], l2_costs, width, label='Layer 2', alpha=0.8, color='green')
    
    ax.set_xlabel('Gas Price (Gwei)', fontsize=12)
    ax.set_ylabel('Total Cost (USD)', fontsize=12)
    ax.set_title(f'Cost Comparison at Different Gas Prices\n({num_transactions} transactions)',
                 fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([f"{gp}" for gp in gas_prices])
    ax.legend(fontsize=11)
    ax.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Gas Price Sensitivity ({num_transactions} transactions):")
    for i, gp in enumerate(gas_prices):
        savings = ((l1_costs[i] - l2_costs[i]) / l1_costs[i]) * 100
        print(f"   {gp:>3} Gwei: L1=${l1_costs[i]:>8.2f}, L2=${l2_costs[i]:>7.2f}, "
              f"Savings={savings:.1f}%")

compare_gas_prices(1000)

**Key Takeaway**: L2 savings increase with transaction volume and remain consistent regardless of gas price fluctuations!

---

# Section 5: Challenge Exercises

Test your understanding by extending the simulations!

## Challenge 1: Three-Party State Channel

**Task**: Extend the `PaymentChannel` class to support 3 parties (Alice, Bob, Charlie).

**Requirements**:
- Each party deposits initial funds
- Support transfers between any two parties
- Maintain correct balances
- Close with final three-way settlement

**Hint**: Modify `ChannelState` to include `charlie_balance` and update `update_state()` logic.

In [ ]:
# YOUR CODE HERE
# Extend ChannelState and PaymentChannel for 3 parties

@dataclass
class ThreePartyChannelState:
    """Represents a three-party channel state."""
    channel_id: str
    alice_balance: float
    bob_balance: float
    charlie_balance: float
    nonce: int
    timestamp: float
    
    # TODO: Implement to_hash() method
    # TODO: Implement __repr__() method

class ThreePartyChannel:
    """Simulates a three-party payment channel."""
    
    def __init__(self, channel_id: str, alice_deposit: float, bob_deposit: float, charlie_deposit: float):
        # TODO: Initialize channel state
        pass
    
    def update_state(self, amount: float, sender: str, receiver: str):
        # TODO: Implement state updates
        # Sender and receiver should be 'alice', 'bob', or 'charlie'
        pass
    
    def close_channel(self):
        # TODO: Implement channel closure
        pass

# Test your implementation:
# three_party = ThreePartyChannel('abc-channel', 10.0, 10.0, 10.0)
# three_party.update_state(2.0, 'alice', 'bob')
# three_party.update_state(1.5, 'charlie', 'alice')
# three_party.close_channel()

### Understanding ZK-Rollups

**What is a ZK-Rollup?** ZK stands for "Zero-Knowledge" - a cryptographic technique that lets you prove something is true without revealing the details.

**Simple Analogy**: Imagine proving you know the password to a safe without telling anyone the password. ZK-Rollups prove transactions are valid without posting all transaction data to the blockchain.

**Key Terms**:
- **SNARK**: Succinct Non-interactive ARgument of Knowledge - a compact proof (think: a small certificate)
- **STARK**: Scalable Transparent ARgument of Knowledge - another proof type that's even more secure
- **Validity Proof**: Mathematical proof that state transitions are correct

**Advantage over Optimistic Rollups**: No 7-day waiting period because validity is proven immediately!

### Understanding ZK-Rollups

**What is a ZK-Rollup?** ZK stands for "Zero-Knowledge" - a cryptographic technique that lets you prove something is true without revealing the details.

**Simple Analogy**: Imagine proving you know the password to a safe without telling anyone the password. ZK-Rollups prove transactions are valid without posting all transaction data to the blockchain.

**Key Terms**:
- **SNARK**: Succinct Non-interactive ARgument of Knowledge - a compact proof (think: a small certificate)
- **STARK**: Scalable Transparent ARgument of Knowledge - another proof type that's even more secure
- **Validity Proof**: Mathematical proof that state transitions are correct

**Advantage over Optimistic Rollups**: No 7-day waiting period because validity is proven immediately!

## Challenge 2: Simple ZK-Rollup Concept

**Task**: Implement a simplified ZK-rollup that posts state commitments without full transaction data.

**Requirements**:
- Batch transactions and compute state root
- Post ONLY state root to L1 (not transaction data)
- Simulate "validity proof" (hash commitment)
- Calculate even greater compression than optimistic rollup

**Hint**: Instead of posting transaction data, just post the state root hash.

In [ ]:
# YOUR CODE HERE
# Implement ZKRollup class

class ZKRollup:
    """Simulates ZK-rollup with validity proofs."""
    
    def __init__(self):
        # TODO: Initialize rollup state
        pass
    
    def add_transaction(self, tx: dict):
        # TODO: Add transaction to batch
        pass
    
    def generate_proof(self) -> dict:
        # TODO: Generate validity proof (simplified: hash commitment)
        # Should return proof that state transition is valid
        pass
    
    def post_to_l1(self, proof: dict) -> dict:
        # TODO: Post only proof to L1 (no transaction data!)
        # Calculate data size and compare to optimistic rollup
        pass

# Test your implementation:
# zk_rollup = ZKRollup()
# for i in range(10):
#     zk_rollup.add_transaction({'from': f'user{i}', 'to': f'user{i+1}', 'amount': 1.0})
# proof = zk_rollup.generate_proof()
# zk_rollup.post_to_l1(proof)

## Challenge 3: Cross-Rollup Bridge

**Task**: Simulate asset transfers between two different rollups.

**Requirements**:
- Create two separate rollup instances
- Implement bridge contract that locks assets on Rollup A and mints on Rollup B
- Support bidirectional transfers
- Track locked/minted balances

**Hint**: Bridge needs to maintain total locked balance equal to total minted.

In [ ]:
# YOUR CODE HERE
# Implement RollupBridge class

class RollupBridge:
    """Simulates asset bridge between two rollups."""
    
    def __init__(self, rollup_a: SimpleRollup, rollup_b: SimpleRollup):
        self.rollup_a = rollup_a
        self.rollup_b = rollup_b
        # TODO: Initialize bridge state
        pass
    
    def transfer_a_to_b(self, user: str, amount: float):
        # TODO: Lock assets on rollup A, mint on rollup B
        pass
    
    def transfer_b_to_a(self, user: str, amount: float):
        # TODO: Burn assets on rollup B, unlock on rollup A
        pass
    
    def verify_invariant(self) -> bool:
        # TODO: Verify that locked_on_a == minted_on_b
        pass

# Test your implementation:
# rollup_a = SimpleRollup()
# rollup_b = SimpleRollup()
# bridge = RollupBridge(rollup_a, rollup_b)
# bridge.transfer_a_to_b('alice', 10.0)
# bridge.transfer_b_to_a('alice', 5.0)
# print(bridge.verify_invariant())

---

# Summary and Key Takeaways

## What We Learned

1. **State Channels**: Off-chain scaling for bilateral interactions
   - Unlimited updates between two parties
   - Only 2 L1 transactions (open/close)
   - Best for: Payment networks, gaming, frequent interactions

2. **Rollups**: Batch execution with L1 data availability
   - 10-100x cost reduction
   - Compress many transactions into one L1 batch
   - Best for: General-purpose smart contracts

3. **Optimistic Rollups**: Fraud proof security model
   - Assume validity, allow challenges
   - 7-day withdrawal delay
   - Examples: Arbitrum, Optimism

4. **ZK-Rollups**: Validity proof security model
   - Cryptographic proofs of correctness
   - Instant finality
   - Examples: zkSync, StarkNet

## Gas Savings Recap

- **State Channels**: ~95%+ savings (after setup)
- **Optimistic Rollups**: ~90-95% savings
- **ZK-Rollups**: ~95-99% savings

## Trade-offs

| Solution | Throughput | Finality | Complexity | Security Model |
|----------|------------|----------|------------|----------------|
| State Channels | High | Instant | Low | Bilateral | 
| Optimistic Rollups | High | Delayed (7d) | Medium | Fraud proofs |
| ZK-Rollups | Very High | Instant | High | Validity proofs |

## Real-World Adoption

- **Arbitrum**: $2B+ TVL, Optimistic rollup
- **Optimism**: $1B+ TVL, Optimistic rollup
- **zkSync**: $100M+ TVL, ZK-rollup
- **Polygon zkEVM**: ZK-rollup with EVM compatibility
- **Lightning Network**: State channels for Bitcoin

**The Future**: Layer 2 is becoming the primary scaling solution for Ethereum and other blockchains!

---

# Additional Resources

**Documentation**:
- [Ethereum L2 Scaling Solutions](https://ethereum.org/en/developers/docs/scaling/)
- [Vitalik's Rollup Guide](https://vitalik.ca/general/2021/01/05/rollup.html)
- [Arbitrum Documentation](https://developer.arbitrum.io/)
- [zkSync Documentation](https://docs.zksync.io/)

**Research Papers**:
- [SoK: Layer-Two Blockchain Protocols](https://eprint.iacr.org/2019/360.pdf)
- [Fraud and Data Availability Proofs](https://arxiv.org/abs/1809.09044)

**Tools**:
- [L2BEAT](https://l2beat.com/) - L2 analytics and metrics
- [L2 Fees](https://l2fees.info/) - Real-time L2 gas cost comparison

---

**[END OF NOTEBOOK]**